In [1]:
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import string
from itertools import islice
from selenium.webdriver.common.keys import Keys
# from Feed import Feed
from icecream import ic
from typing import Iterator, Any, List

In [2]:
def get_script(fname:str):
  script = ''
  with open(f'./scripts/{fname}.js','r') as f:
    script = f.read()
  script = script.replace('\n','')
  def replacer(*args):
    acc = script
    for i,x in enumerate(args):
      acc = acc.replace('#'+str(i),str(x))
    return 'try{' + f'console.log(`{fname}`,JSON.parse(`{json.dumps(args)}`));'+acc+'}catch(err){console.log(`{fname}`,err);}'
  return replacer
def script_runner(driver):
  def run_script(name:str,verbose=False):
    def script(*args:Any):
      script_string = get_script(name)(*args)
      answer = driver.execute_script(script_string)
      if verbose:
        print('script_string: ',script_string,' Answer: ', answer)
      return answer
    return script
  return run_script

In [3]:
from Surfer import GetBrowser,RedditFeed,Feed

driver = GetBrowser()

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:25<00:00, 282kB/s] 


In [4]:
run_script = script_runner(driver)

In [5]:
def uuid():
    letters_digits = string.ascii_letters
    return ''.join(random.choice(letters_digits) for i in range(20))

In [6]:
def N(
  mean = 0.5
	, stddev = 0.01
	, bounds = None
):
	if bounds:
		return max(bounds[0], min(random.normalvariate(mean, stddev) , bounds[1]))
	else:
		return random.normalvariate(mean, stddev)


In [19]:
class RedditFeed(Feed):
	def __init__(self, driver,skip=True,verbose=False):
		super()
		self.name = 'Reddit Feed'
		# Store the WebDriver instance
		self.driver = driver
		self.verbose = verbose
		self.in_feed = True
		if not skip:
			self.driver.get('https://www.reddit.com/')
			run_script('listen to clicks')()

		run_script('init_reddit',verbose=self.verbose)()

	def __iter__(self) -> Iterator[Any]:
		while True and self.in_feed:
			if not run_script('check cursor',verbose=self.verbose)():
				continue
			run_script('scroll 2 cursor',verbose=self.verbose)(N(bounds=[0,1]),N(bounds=[0,1]),random.randint(1,25)/100)
			while run_script('check still scrolling')():
				time.sleep(0.1)
			yield run_script('parse post',verbose=self.verbose)()
			run_script('cursor next')()

	def click_post(self, newTab=True):
		id = uuid()
		run_script('mark cursor', verbose=self.verbose)(id)
		# Find the DOM element to click on
		sel = f'.{id}'
		element = driver.find_element(By.CSS_SELECTOR, sel)
		# Get the dimensions and position of the element
		element.click()
		self.in_feed = False
		return element


	def close(self):
		# Close the WebDriver instance
		self.driver.quit()

In [20]:
rf = RedditFeed(driver,skip=False)
# rf = RedditFeed(driver,skip=True)

In [21]:
time.sleep(4)
for x in islice(rf,15):
  print(x)
  time.sleep(2*max(1,len(x) * 0.01))

{'comments': '143 Comments', 'excerpt': None, 'links': [{'link': 'https://www.reddit.com/r/france/', 'text': '143 Comments'}, {'link': 'https://www.reddit.com/r/france/', 'text': ''}, {'link': 'https://www.reddit.com/user/FrankMaleir/', 'text': 'bfmtv.com/politi...'}, {'link': 'https://www.reddit.com/r/france/comments/12ya3eb/pour_7_fran%C3%A7ais_sur_10_la_r%C3%A9%C3%A9lection_demmanuel/', 'text': 'Société'}, {'link': 'https://www.reddit.com/r/france/?f=flair_name%3A%22Soci%C3%A9t%C3%A9%22', 'text': 'Pour 7 Français sur 10, la réélection d’Emmanuel Macron a été une mauvaise chose pour le pays'}, {'link': 'https://www.bfmtv.com/politique/elysee/pour-7-francais-sur-10-la-reelection-d-emmanuel-macron-a-ete-une-mauvaise-chose-pour-le-pays_AN-202304240627.html', 'text': 'u/FrankMaleir'}, {'link': 'https://www.bfmtv.com/politique/elysee/pour-7-francais-sur-10-la-reelection-d-emmanuel-macron-a-ete-une-mauvaise-chose-pour-le-pays_AN-202304240627.html', 'text': 'r/france'}, {'link': 'https://ww

In [22]:
e = rf.click_post()

In [23]:
e

<selenium.webdriver.remote.webelement.WebElement (session="3369447a1ca0fbee88d35049d616e4e9", element="b1687de4-c5bf-4c0b-9b5a-25111d0857ea")>

In [24]:
run_script('cluster')()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x009BDCE3+50899]
	(No symbol) [0x0094E111]
	(No symbol) [0x00855588]
	(No symbol) [0x00847CB6]
	(No symbol) [0x008479D9]
	(No symbol) [0x00856990]
	(No symbol) [0x008ADB9C]
	(No symbol) [0x0089B6F6]
	(No symbol) [0x00877708]
	(No symbol) [0x0087886D]
	GetHandleVerifier [0x00C23EAE+2566302]
	GetHandleVerifier [0x00C592B1+2784417]
	GetHandleVerifier [0x00C5327C+2759788]
	GetHandleVerifier [0x00A55740+672048]
	(No symbol) [0x00958872]
	(No symbol) [0x009541C8]
	(No symbol) [0x009542AB]
	(No symbol) [0x009471B7]
	BaseThreadInitThunk [0x77D0FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77EB7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77EB7A6E+238]


In [11]:
# decultter html
# Array.from(x.querySelectorAll('*')).map((x,i)=>{
#     const cl = Array.from(x.classList)
#     const to_remove = cl.filter(x=> !/^[a-z-]+$/.exec(x))
#     for (let cls of to_remove) {
#         x.classList.remove(cls)
#     }
#     if (x.className === ''){
#         x.removeAttribute('class')
#     }
# })